# 9 Virtual Memory

Virtual memory is an elegant interaction of hardware exceptions, hardware address translation, main memory, disk files, and kernel software that provides each process with a large, uniform, and private address space.

With one clean mechanism, virtual memory provides three important capabilities: 
1. It uses main memory efficiently by treating it as a cache for an address space stored on disk, keeping only the active areas in main memory and transferring data back and forth between disk and memory as needed.
2. It simplifies memory management by providing each process with a uniform address space.
3. It protects the address space of each process from corruption by other processes.

## 9.1 Physical and Virtual Addressing

The main memory of a computer system is organized as an array of M contiguous byte-size cells. Each byte has a unique _physical address_ (PA). The first byte has an address of 0, the next byte an address of 1, the next byte an address of 2, and so on. Given this simple organization, the most natural way for a CPU to access memory would be to use physical addresses. We call this approach _physical addressing_. **Figure 9.1** shows an example of physical addressing in the context of a load instruction that reads the 4-byte word starting at physical address 4. When the CPU executes the load instruction, it generates an effective physical address and passes it to main memory over the **memory bus**. The **main memory** fetches the 4-byte word starting at physical address 4 and returns it to the CPU, which stores it in a register.

![](asset/ch9/1.png)

With virtual addressing, the CPU accesses main memory by generating a _virtual address (VA)_, which is converted to the appropriate physical address before being sent to main memory. The task of converting a virtual address to a physical one is known as _address translation_. Dedicated hardware on the CPU chip called the _memory management unit (MMU)_ translates virtual addresses **on the fly**, using a lookup table stored in main memory whose contents are managed by the operating system.

![](asset/ch9/2.png)

## 9.2 Address Spaces

An _address space_ is an ordered set of nonnegative integer addresses: $\{0, 1, 2, ...\}$

If the integers in the address space are consecutive, then we say that it is a _linear address space_. To simplify our discussion, we will always assume linear address spaces. In a system with virtual memory, the CPU generates virtual addresses from an address space of $N = 2^n$ addresses called the _virtual address space_: $\{0, 1, 2, ..., N\}$

A system also has a _physical address space_ that corresponds to the $M$ bytes of physical memory in the system: $\{0, 1, 2, ..., M-1\}$, $M$ is not required to be a power of 2, but to simplify the discussion, we will assume that $M = 2^m$.

Each byte of main memory has a virtual address chosen from the virtual address space, and a physical address chosen from the physical address space.


## 9.3 VM as a Tool for Caching

Conceptually, a virtual memory is organized as an array of $N$ contiguous byte-size
cells stored on disk. Each byte has a unique virtual address that serves as an index
into the array. The contents of the array on disk are cached in main memory. As
with any other cache in the memory hierarchy, the data on disk (the lower level)
is partitioned into blocks that serve as the transfer units between the disk and
the main memory (the upper level). VM systems handle this by partitioning the virtual memory into fixed-size blocks called _virtual pages (VPs)_. Each virtual page is $P = 2^p$ bytes in size. Similarly, physical memory is partitioned into _physical pages (PPs)_, also $P$ bytes in size. (Physical pages are also referred to as _page frames_.)

At any point in time, the set of virtual pages is partitioned into three disjoint
subsets:
* _Unallocated._ Pages that have not yet been allocated (or created) by the VM system. Unallocated blocks do not have any data associated with them, and thus do not occupy any space on disk.
* _Cached._ Allocated pages that are currently cached in physical memory.
* _Uncached._ Allocated pages that are not cached in physical memory.

The example in **Figure 9.3** shows a small virtual memory with eight virtual
pages. Virtual pages 0 and 3 have not been allocated yet, and thus do not yet exist
on disk. Virtual pages 1, 4, and 6 are cached in physical memory. Pages 2, 5, and 7
are allocated but are not currently cached in physical memory.

![](asset/ch9/3.png)

### 9.3.1 DRAM Cache Organization

To help us keep the different caches in the memory hierarchy straight, we will use
the term _SRAM cache_ to denote the L1, L2, and L3 cache memories between the
CPU and main memory, and the term _DRAM cache_ to denote the VM system’s
cache that caches virtual pages in main memory.

Because of the large miss penalty and the expense of accessing the first byte from disk, virtual pages tend to be large—typically 4 KB to 2 MB. Due to the large miss penalty, DRAM caches are fully associative; **that is, any virtual page can be placed in any physical page**.

The replacement policy on misses also assumes greater importance, because the penalty associated with replacing the wrong virtual page is so high. Thus, operating systems use much more sophisticated replacement algorithms for DRAM caches than the hardware does for SRAM caches. Finally, because of the large access time of disk, DRAM caches always use write-back instead of write-through.

### 9.3.2 Page Tables

As with any cache, the VM system must have some way to determine if a virtual
page is cached somewhere in DRAM. If so, the system must determine which
physical page it is cached in. If there is a miss, the system must determine where the virtual page is stored on disk, select a victim page in physical memory,
and copy the virtual page from disk to DRAM, replacing the victim page.

These capabilities are provided by a combination of operating system soft-
ware, address translation hardware in the MMU (memory management unit), and
a data structure stored in physical memory known as a page table that maps vir-
tual pages to physical pages. The address translation hardware reads the page table
each time it converts a virtual address to a physical address. The operating system
is responsible for maintaining the contents of the page table and transferring pages
back and forth between disk and DRAM.

**Figure 9.4** shows the basic organization of a page table. A page table is an array
of _page table entries (PTEs)_. Each page in the virtual address space has a PTE at
a fixed offset in the page table. For our purposes, we will assume that each PTE
consists of a valid bit and an n-bit address field. The valid bit indicates whether
the virtual page is currently cached in DRAM. If the valid bit is set, the address
field indicates the start of the corresponding physical page in DRAM where the
virtual page is cached. If the valid bit is not set, then a null address indicates that
the virtual page has not yet been allocated. Otherwise, the address points to the
start of the virtual page on disk.
An important point to notice about **Figure 9.4** is that because the DRAM
cache is fully associative, any physical page can contain any virtual page.

![](asset/ch9/4.png)

### 9.3.3 Page Hits

Consider what happens when the CPU reads a word of virtual memory contained in VP 2, which is cached in DRAM (**Figure 9.5**). Using a technique we will describe in detail in **Section 9.6**, the address translation hardware uses the virtual address as an index to locate PTE 2 and read it from memory. Since the valid bit is set, the address translation hardware knows that VP 2 is cached in memory. So it uses the physical memory address in the PTE (which points to the start of the cached page in PP 1) to construct the physical address of the word.

![](asset/ch9/5.png)

### 9.3.4 Page Faults

In virtual memory parlance, a DRAM cache miss is known as a _page fault_. **Figure 9.6** shows the state of our example page table before the fault. The CPU has
referenced a word in VP 3, which is not cached in DRAM. The address transla-
tion hardware reads PTE 3 from memory, infers from the valid bit that VP 3 is
not cached, and triggers a page fault exception.

The page fault exception invokes
a page fault exception handler in the kernel, which selects a victim page—in this
case, VP 4 stored in PP 3. If VP 4 has been modified, then the kernel copies it back
to disk. In either case, the kernel modifies the page table entry for VP 4 to reflect
the fact that VP 4 is no longer cached in main memory.

Next, the kernel copies VP 3 from disk to PP 3 in memory, updates PTE 3,
and then returns. When the handler returns, it restarts the faulting instruction,
which resends the faulting virtual address to the address translation hardware.
But now, VP 3 is cached in main memory, and the page hit is handled normally by
the address translation hardware. **Figure 9.7** shows the state of our example page
table after the page fault.

Virtual memory was invented in the early 1960s, long before the widening
CPU-memory gap spawned SRAM caches. As a result, virtual memory systems
use a different terminology from SRAM caches, even though many of the ideas
are similar. In virtual memory parlance, blocks are known as pages. The activity
of transferring a page between disk and memory is known as _swapping or paging_.
Pages are _swapped in (paged in)_ from disk to DRAM, and _swapped out (paged
out)_ from DRAM to disk. The strategy of waiting until the last moment to swap in a page, when a miss occurs, is known as _demand paging_.

### 9.3.5 Allocating Pages

**Figure 9.8** shows the effect on our example page table when the operating system
allocates a new page of virtual memory—for example, as a result of calling malloc.
In the example, VP 5 is allocated by creating room on disk and updating PTE 5
to point to the newly created page on disk.

![](asset/ch9/6.png)



### 9.3.6 Locality to the Rescue Again

Although the total number of distinct pages that programs reference during an
entire run might exceed the total size of physical memory, the principle of locality
promises that at any point in time they will tend to work on a smaller set of active
pages known as the _working set_ or _resident set_. After an initial overhead where
the working set is paged into memory, subsequent references to the working set
result in hits, with no additional disk traffic.

As long as our programs have good temporal locality, virtual memory systems
work quite well. But of course, not all programs exhibit good temporal locality. If
the working set size exceeds the size of physical memory, then the program can
produce an unfortunate situation known as _thrashing_, where pages are swapped in
and out continuously. Although virtual memory is usually efficient, if a program’s
performance slows to a crawl, the wise programmer will consider the possibility
that it is thrashing.


## 9.4 VM as a Tool for Memory Management

Notice that multiple virtual pages can be mapped to
the same shared physical page.

![](asset/ch9/7.png)

The combination of demand paging and separate virtual address spaces has
a profound impact on the way that memory is used and managed in a system. In
particular, VM simplifies linking and loading, the sharing of code and data, and
allocating memory to applications.

* _Simplifying linking._ A separate address space allows each process to use the same basic format for its memory image, regardless of where the code and data actually reside in physical memory.

* _Simplifying loading._ Virtual memory also makes it easy to load executable and shared object files into memory. To load the `.text` and `.data` sections of an object file into a newly created process, the Linux loader allocates virtual pages for the code and data segments, marks them as invalid (i.e., not cached), and points their page table entries to the appropriate locations in the object file. The interesting point is that the loader never actually copies any data from disk into memory. The data are paged in automatically and on demand by the virtual memory system the first time each page is referenced, either by the CPU when it fetches an instruction or by an executing instruction when it references a memory location.

* _Simplifying sharing._ Separate address spaces provide the operating system with a consistent mechanism for managing sharing between user processes and the operating system itself. In general, each process has its own private code, data, heap, and stack areas that are not shared with any other process. In this case, the operating system creates page tables that map the corresponding virtual pages to disjoint physical pages. However, in some instances it is desirable for processes to share code and data. For example, every process must call the same operating system kernel code, and every C program makes calls to routines in the standard C library such as printf. Rather than including separate copies of the kernel and standard C library in each process, the operating system can arrange for multiple processes to share a single copy of this code by mapping the appropriate virtual pages in different processes to the same physical pages.

* _Simplifying memory allocation._ Virtual memory provides a simple mechanism
for allocating additional memory to user processes. When a program running
in a user process requests additional heap space (e.g., as a result of calling
`malloc`), the operating system allocates an appropriate number, say, $k$, of
contiguous virtual memory pages, and maps them to $k$ arbitrary physical pages
located anywhere in physical memory. Because of the way page tables work,
there is no need for the operating system to locate $k$ contiguous pages of
physical memory. The pages can be scattered randomly in physical memory.

## 9.5 VM as a Tool for Memory Protection

Any modern computer system must provide the means for the operating system
to control access to the memory system. A user process should not be allowed to modify its read-only code section. Nor should it be allowed to read or modify
any of the code and data structures in the kernel. It should not be allowed to read
or write the private memory of other processes, and it should not be allowed to
modify any virtual pages that are shared with other processes, unless all parties
explicitly allow it (via calls to explicit interprocess communication system calls).

As we have seen, providing separate virtual address spaces makes it easy to
isolate the private memories of different processes. But the address translation
mechanism can be extended in a natural way to provide even finer access control.
Since the address translation hardware reads a PTE each time the CPU generates
an address, it is straightforward to control access to the contents of a virtual page
by adding some additional permission bits to the PTE. **Figure 9.10** shows the
general idea.

![](asset/ch9/8.png)

In this example, we have added three permission bits to each PTE. The SUP bit
indicates whether processes must be running in kernel (supervisor) mode to access
the page. Processes running in kernel mode can access any page, but processes
running in user mode are only allowed to access pages for which SUP is 0. The
READ and WRITE bits control read and write access to the page. For example,
if process i is running in user mode, then it has permission to read VP 0 and to
read or write VP 1. However, it is not allowed to access VP 2.

If an instruction violates these permissions, then the CPU triggers a general
protection fault that transfers control to an exception handler in the kernel, which
sends a SIGSEGV signal to the offending process. Linux shells typically report this
exception as a “segmentation fault.”

## 9.6 Address Translation

![](asset/ch9/9.png)

In this section, we are omitting a number of details, especially related to timing, that are important to hardware designers but are beyond our scope. For your
reference, **Figure 9.11** summarizes the symbols that we will be using throughout
this section.

Address translation is a mapping between the elements of an N-
element virtual address space (VAS) and an M-element physical address space
(PAS).

**Figure 9.12** shows how the MMU uses the page table to perform this mapping.
A control register in the CPU, the page table base register (PTBR) points to the
current page table. The $n$-bit virtual address has two components: a $p$-bit virtual
page offset (VPO) and an $(n − p)$-bit virtual page number (VPN). The MMU uses
the VPN to select the appropriate PTE. For example, VPN 0 selects PTE 0, VPN 1
selects PTE 1, and so on. The corresponding physical address is the concatenation
of the physical page number (PPN) from the page table entry and the VPO from
the virtual address. **Notice that since the physical and virtual pages are both P
bytes, the physical page offset (PPO) is identical to the VPO.**

![](asset/ch9/10.png)

**Figure 9.13(a)** shows the steps that the CPU hardware performs when there
is a page hit.
* _Step 1._ The processor generates a virtual address and sends it to the MMU.
* _Step 2._ The MMU generates the PTE address and requests it from the cache/
main memory.
* _Step 3._ The cache/main memory returns the PTE to the MMU.
* _Step 4._ The MMU constructs the physical address and sends it to the cache/main
memory.
* _Step 5._ The cache/main memory returns the requested data word to the pro-
cessor.

Unlike a page hit, which is handled entirely by hardware, handling a page
fault requires cooperation between hardware and the operating system kernel
(**Figure 9.13(b)**).

* _Steps 1 to 3._ The same as steps 1 to 3 in Figure 9.13(a).
* _Step 4._ The valid bit in the PTE is zero, so the MMU triggers an exception,
which transfers control in the CPU to a page fault exception handler in
the operating system kernel.
* _Step 5._ The fault handler identifies a victim page in physical memory, and if that
page has been modified, pages it out to disk.
* _Step 6._ The fault handler pages in the new page and updates the PTE in memory.
* _Step 7._ The fault handler returns to the original process, causing the faulting
instruction to be restarted. The CPU resends the offending virtual address
to the MMU. Because the virtual page is now cached in physical memory,
there is a hit, and after the MMU performs the steps in **Figure 9.13(a)**, the
main memory returns the requested word to the processor.

![](asset/ch9/11.png)

首先，处理器生成 VA (Virtual address)并发送给 MMU (Memory management unit)，虚拟地址去除 VPO (Virtual page offset，同时也等于 Physical page offset) 后便是对应的 VPN (Virtual page number)，根据 VPN 和 page table 的地址可以 生成 PTEA (Page table entries address)。随后 MMU 根据 PTEA 到存储在内存中的 page table 中取得 PTE。PTE 中存储了 physical page number (Cached) or disk address (Not cached)。
* 如果 valid bit is set，则储存的是 physical page number，则可以直接生成 PA (Physical address)，从 main memory 中获取 data。__这种情况也被称为 page hit__。
* 如果 valid bit is not set 时，分为两种情况，如果此时 PTE 储存值为 NULL，则说明page is not allocated，说明此时程序员是在 access 没有被 allocate 的内存区域，结果自然就是 segfault。正确的方式是，1) 不使用未收集的内存 2) 使用 malloc 等函数先收集内存。
* 如果 valid bit is not set 时，PTE 储存值非 NULL，说明此时 PTE 中存储的是 disk address。此时 VP (Virtual page) 未在 main memory 中，__这种情况也被称为 page miss__，会触发内核的 page fault exception handler，handler 会选择一个 main memory 中的 victim page，如果选中的 victim page 已经被 modified，那么内核需要把它写回 disk。原本映射到 victim page 的 PTE 的 valid bit is unset (Uncached)。最后，内核把 disk address 中的数据复制到 victim page 中，set valid bit。

### 9.6.1 Integrating Caches and VM

In any system that uses both virtual memory and SRAM caches, there is the issue of whether to use virtual or physical addresses to access the SRAM cache. Although a detailed discussion of the trade-offs is beyond our scope here, most systems opt for physical addressing. With physical addressing, it is straightforward for multiple processes to have blocks in the cache at the same time and to share blocks from the same virtual pages. Further, the cache does not have to deal with protection issues, because access rights are checked as part of the address translation process.

**Figure 9.14** shows how a physically addressed cache might be integrated with virtual memory. The main idea is that the address translation occurs before the cache lookup. Notice that page table entries can be cached, just like any other data words.

![](asset/ch9/12.png)

### 9.6.2 Speeding Up Address Translation with a TLB

As we have seen, every time the CPU generates a virtual address, the MMU must refer to a PTE in order to translate the virtual address into a physical address. In the worst case, this requires an additional fetch from memory, at a cost of tens to hundreds of cycles. If the PTE happens to be cached in L1, then the cost goes down to a handful of cycles. However, many systems try to eliminate even this cost by including a **small cache of PTEs in the MMU** called a _translation lookaside buffer (TLB)_.

![](asset/ch9/13.png)

A TLB is a small, virtually addressed cache where each line holds a block
consisting of a single PTE. A TLB usually has a high degree of associativity. As shown in **Figure 9.15**, the index and tag fields that are used for set selection and line matching are extracted from the virtual page number in the virtual address. If the TLB has $T = 2^t$ sets, then the _TLB index (TLBI)_ consists of the t least significant bits of the VPN, and the _TLB tag (TLBT)_ consists of the remaining bits in the VPN.